In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout , Conv2D , Flatten , Dense
from keras import layers, models
import os



In [5]:
# Check number of classes
train_dir = 'OBJECTS/train'
num_classes = len(os.listdir(train_dir))
print(f"Number of classes: {num_classes}")

Number of classes: 6


In [6]:

# Define the data augmentation configuration for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define the data augmentation configuration for validation data
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and augment images from the training directory
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to training directory
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

# Load images from the validation directory
validation_generator = validation_datagen.flow_from_directory(
    'OBJECTS/test',  # Path to validation directory
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)






Found 2527 images belonging to 6 classes.
Found 610 images belonging to 6 classes.


In [7]:

# Build the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(layers.Dense(num_classes, activation='softmax'))  # Adjust the output layer for multi-class classification








model.save('GARBAGE_CLASSIFIER.h5')

c:\Users\AMAN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for multi-class classification
              optimizer='rmsprop',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator)

Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 35s 419ms/step - accuracy: 0.2224 - loss: 1.7990 - val_accuracy: 0.3869 - val_loss: 1.5586
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 33s 404ms/step - accuracy: 0.3329 - loss: 1.6362 - val_accuracy: 0.3213 - val_loss: 1.6987
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 32s 390ms/step - accuracy: 0.3580 - loss: 1.5409 - val_accuracy: 0.4803 - val_loss: 1.3475
Epoch 4/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 33s 403ms/step - accuracy: 0.4134 - loss: 1.4326 - val_accuracy: 0.4557 - val_loss: 1.3426
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 36s 441ms/step - accuracy: 0.4388 - loss: 1.3607 - val_accuracy: 0.3705 - val_loss: 1.5806
Epoch 6/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 31s 384ms/step - accuracy: 0.4784 - loss: 1.3401 - val_accuracy: 0.4213 - val_loss: 1.3795
Epoch 7/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 31s 385ms/step - accuracy: 0.5180 - loss: 1.2327 - val_accuracy: 0.5049 - val_loss: 1.1884
Epoch 8/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 32s 390ms/step - accuracy: 0.5351 - loss: 1.1923 - val_accu

In [10]:
model.save("GARBAGE_CLASSIFIER.h5")